In [2]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


## INICIAR MODELO

In [13]:
model = pyo.ConcreteModel()

#parametros
capacidade_fabrica = {
    'F1':350,
    'F2':600
}

demanda_cd = {
    'CD1':300,
    'CD2':400,
    'CD3':250
}

#De Para CD1,Cd2,Cd3
custo_transporte = {
    'F1':[8,6,10],
    'F2':[9,12,13]
}

#Definir SET
model.FABRICAS = pyo.Set(initialize=['F1','F2'])
model.CD = pyo.Set(initialize=['CD1','CD2','CD3'])

#Variaveis
model.x = pyo.Var(model.FABRICAS,model.CD, domain=NonNegativeIntegers)

#Restrições de quantidade
def soma_capacidade_fabricas(model, f):
    return sum (model.x[f,c] for c in model.CD) <= capacidade_fabrica[f]
model.soma_capacidade_fabricas = pyo.Constraint(model.FABRICAS, rule=soma_capacidade_fabricas)

def soma_capacidade_cd(model, c):
    return sum(model.x[f,c] for f in model.FABRICAS) <= demanda_cd[c]
model.soma_capacidade_cd= pyo.Constraint(model.CD, rule=soma_capacidade_cd)

#Objetivo
def objetivo1(model):
    
    custo_total_fabrica = sum(sum(model.x[f,c] * custo_transporte[c] for c in model.CD)for f in model.FABRICAS)
    # custo_total_f2 = sum(model.x[f,c] * custo_transporte[c] for c in model.CD)

    return custo_total_fabrica
model.objetivo = pyo.Objective(rule=objetivo1, sense=pyo.minimize)



ERROR: Rule failed when generating expression for Objective objetivo with
index None: KeyError: 'CD1'
ERROR: Constructing component 'objetivo' from data=None failed:
        KeyError: 'CD1'


KeyError: 'CD1'

In [12]:
model.pprint()

2 Set Declarations
    CD : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'CD1', 'CD2', 'CD3'}
    FABRICAS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'F1', 'F2'}

1 Var Declarations
    x : Size=6, Index=FABRICAS*CD
        Key           : Lower : Value : Upper : Fixed : Stale : Domain
        ('F1', 'CD1') :     0 :  None :  None : False :  True : NonNegativeIntegers
        ('F1', 'CD2') :     0 :  None :  None : False :  True : NonNegativeIntegers
        ('F1', 'CD3') :     0 :  None :  None : False :  True : NonNegativeIntegers
        ('F2', 'CD1') :     0 :  None :  None : False :  True : NonNegativeIntegers
        ('F2', 'CD2') :     0 :  None :  None : False :  True : NonNegativeIntegers
        ('F2', 'CD3') :     0 :  None :  None : False :  True : NonNegativeIntegers

2 Constraint Declarations
    soma_capacidad

In [152]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 180 
results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\DECIV\AppData\Local\Temp\tmpn8kni2vm.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> Problem 'C:\Users\DECIV\AppData\Local\Temp\tmpsj31qxci.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> MIP start file 'C:\Users\DECIV\AppData\Local\Temp\tmpjm_bdqmd.cplex.mst' read.
CPLEX> Problem name         : C:\Users\DECIV\AppData\Local\Temp\tmpsj31qxci.pyomo.lp
Objective sense      : Maximize
Variables            :       8  [Box: 2,  General Integer: 6]
Objective nonzeros   :       2
Linear constraints   :       9  [Less: 7,  Equal: 2]
  Nonzeros           :     

In [ ]:
for i in model.:
    for r in model.:
        print('----------')
        print('----------')
        print(f"Valor de x{i,r}: ",pyo.value(model.x[i,r]))
        print('------')        
        



----------
----------
Valor de x('Milho', 'A'):  0.0
------
Producao total de A 0.0
----------
----------
Valor de x('Milho', 'B'):  0.0
------
Producao total de B 0.0
----------
----------
Valor de x('Soja', 'A'):  0.0
------
Producao total de A 0.0
----------
----------
Valor de x('Soja', 'B'):  0.0
------
Producao total de B 0.0
----------
----------
Valor de x('Suplemento', 'A'):  0.0
------
Producao total de A 0.0
----------
----------
Valor de x('Suplemento', 'B'):  0.0
------
Producao total de B 0.0


In [98]:
model.pprint()

2 Set Declarations
    INGREDIENTES : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Milho', 'Soja', 'Suplemento'}
    RACAO : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'A', 'B'}

2 Var Declarations
    producao : Size=2, Index=RACAO
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   0.0 :  None : False : False : NonNegativeReals
          B :     0 :   0.0 :  None : False : False : NonNegativeReals
    x : Size=6, Index=INGREDIENTES*RACAO
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
             ('Milho', 'A') :     0 :   0.0 :  None : False : False : NonNegativeReals
             ('Milho', 'B') :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Soja', 'A') :     0 :   0.0 :  None : False : False : NonNegativeReals
              ('Soja',